This file is to run and test the NERmodel_trainer function.

Run the below cells to create a training data from a sample text data and train a custom NER.

In [1]:
# Import libraries
from NERmodel_trainer import build_TrainData, train_costume_NER

from zipfile import ZipFile
from io import BytesIO
import requests
import pandas as pd

In [ ]:
# Read the sample text data
with open('sample_train_text.txt', 'r') as f:
    train_text = f.read()

# Download the zip file contents of US-GAAP taxonomy in binary format
url = 'https://www.fasb.org/cs/BlobServer?blobkey=id&blobnocache=true&blobwhere=1175836290847&blobheader=application%2Fzip&blobheadername2=Content-Length&blobheadername1=Content-Disposition&blobheadervalue2=9013838&blobheadervalue1=attachment%3B+filename%3DUS_GAAP_Taxonomy_2021.zip&blobcol=urldata&blobtable=MungoBlobs'
r = requests.get(url)

# Read the content of the zip file as a pandas dataframe
with ZipFile(BytesIO(r.content), 'r') as zipObj:
    file = zipObj.open(zipObj.namelist()[0])
    tax_df = pd.read_excel(file)

# Remove depricated taxonomy items
tax_df = tax_df[tax_df['deprecatedDate'].isna()]

# Create a list of labels as entity labels for training the NER
pattern = '\[.*\]'
tax_df['label'] = tax_df['label'].str.replace(pattern, '').tolist()
Labels = tax_df[['name', 'label']]

In [6]:
# Call the function to initiate the training data builder
data_builder = build_TrainData(train_text=train_text, model='en_core_web_md')

# Create the training data
train_data = data_builder.TrainData(Labels)

/home/amin/NLP_app/venv/lib/python3.8/site-packages/spaczz/matcher/tokenmatcher.py:115: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = matcher(doc)


   0 sentences processed;    0 training samples saved.
  50 sentences processed;    5 training samples saved.
 100 sentences processed;    6 training samples saved.
 150 sentences processed;   12 training samples saved.
 200 sentences processed;   13 training samples saved.
 250 sentences processed;   16 training samples saved.
 300 sentences processed;   20 training samples saved.


In [10]:
# Train the spaCy nlp model on the created training dataset
trained_NER = train_costume_NER(train_data=train_data, model='en_core_web_md', 
                                n_iter=10, output_dir='Trained_NER')

Loaded en_core_web_md model.
Iter: 1 - losses: {'ner': 71.91662378197704}
Iter: 2 - losses: {'ner': 45.49556114048783}
Iter: 3 - losses: {'ner': 34.997234170665706}
Iter: 4 - losses: {'ner': 32.64720423617304}
Iter: 5 - losses: {'ner': 32.05719015190242}
Iter: 6 - losses: {'ner': 30.778474935068896}
Iter: 7 - losses: {'ner': 27.555528637208788}
Iter: 8 - losses: {'ner': 25.212736804005942}
Iter: 9 - losses: {'ner': 25.778193587854584}
Iter: 10 - losses: {'ner': 23.435328318111267}
Saved model to  Trained_NER
